## Build the structured data files from the raw data:

In [1]:
from fpl_predictor.build_data import DataBuilder

# Run this code to update the structured data:
db = DataBuilder()

# Run this to access the object without building files:
db = DataBuilder(build_player_files=False, build_master_csv=False)

# Example function:
db.load_year_players(2019).head()

Making master player files ... ✓
Making master player gameweek CSV file ... ✓


,name,code,element_type,first_name,id,second_name,squad_number,team,team_code,web_name
0,Shkodran Mustafi,69140,2,Shkodran,1,Mustafi,None,1,3,Mustafi
1,Héctor Bellerín,98745,2,Héctor,2,Bellerín,None,1,3,Bellerín
2,Sead Kolasinac,111457,2,Sead,3,Kolasinac,None,1,3,Kolasinac
3,Ainsley Maitland-Niles,154043,2,Ainsley,4,Maitland-Niles,None,1,3,Maitland-Niles
4,Sokratis Papastathopoulos,39476,2,Sokratis,5,Papastathopoulos,None,1,3,Sokratis


## Get information about players:

In [2]:
from fpl_predictor.player_information import PlayerInformation
pi = PlayerInformation()
pi.search_players("rui", "patr")

{38533: 'Rui Pedro dos Santos Patrício'}

In [3]:
# Get more details from a specific gameweek:
code = 38533
pi.get_player(code, year=2019, week=5)

{'code': 38533,
 'name': 'Rui Pedro dos Santos Patrício',
 'position': 'GK',
 'value': 51,
 'team': 'Wolves'}

## Build a Squad:

In [6]:
# Use this object to build a squad:
from fpl_predictor.squad_builder import SquadBuilder

# Scoring metric is the column from the data that you want to rank players by:
sqb = SquadBuilder(scoring_metric="ict_index")

In [7]:
codes = [38533, 39487, 149484, 206325, 172780, 103955, 118748, 61366, 44699, 
         57127, 193111, 98747, 109533, 80146, 153723]
sqb.squad.remove_all_players()
for code in codes:
    sqb.add_player(code=code, year=2019, week=5)

Player added to squad: Rui Pedro dos Santos Patrício
Player added to squad: Erik Pieters
Player added to squad: Tyrone Mings
Player added to squad: Oleksandr Zinchenko
Player added to squad: James Maddison
Player added to squad: Raheem Sterling
Player added to squad: Mohamed Salah
Player added to squad: Kevin De Bruyne
Player added to squad: Ashley Barnes
Player added to squad: Teemu Pukki
Player added to squad: Todd Cantwell
Player added to squad: Nick Pope
Player added to squad: Emerson Palmieri dos Santos
Player added to squad: Jordan Ayew
Player added to squad: John Lundstram


In [6]:
# Save a squad to CSV:
sqb.squad.save_squad("gw5")

Squad of 15 players saved at: C:\Users\toby\toby-local\gh\fpl-predictor\data\squads\gw5.csv


In [8]:
# Load a squad from CSV:
sqb.squad.remove_all_players()
sqb.squad.load_squad("gw6")

Squad of 15 players loaded from /Users/tp/projects/fpl-predictor/data/squads/gw6.csv


In [14]:
# See the first team chosen by the scoring_metric:
sqb.squad.first_team

,code,name,position,score,score_per_value,team,value
0,98747,Nick Pope,GK,3.1,0.067391,Burnley,46.0
1,103955,Raheem Sterling,MID,17.3,0.140650,Man City,123.0
2,118748,Mohamed Salah,MID,15.9,0.127200,Liverpool,125.0
3,193111,Todd Cantwell,MID,9.2,0.191667,Norwich,48.0
4,57127,Teemu Pukki,FWD,9.0,0.128571,Norwich,70.0
5,80146,Jordan Ayew,FWD,6.0,0.117647,Crystal Palace,51.0
6,93100,Jannik Vestergaard,DEF,5.4,0.103846,Southampton,52.0
7,206325,Oleksandr Zinchenko,DEF,4.8,0.085714,Man City,56.0
8,61366,Kevin De Bruyne,MID,4.8,0.048980,Man City,98.0
9,39487,Erik Pieters,DEF,3.5,0.074468,Burnley,47.0


In [15]:
sqb.squad.substitutes

,code,name,position,value,score,score_per_value,team
0,153723,John Lundstram,DEF,42,2.5,0.059524,Sheffield Utd
1,149484,Tyrone Mings,DEF,46,1.8,0.039130,Aston Villa
2,38533,Rui Pedro dos Santos Patrício,GK,51,1.0,0.019608,Wolves
3,44699,Ashley Barnes,FWD,67,0.2,0.002985,Burnley


## Captain & Vice-Captain Picks

In [9]:
sqb.squad.captain

{103955: 'Raheem Sterling'}

In [10]:
sqb.squad.vice_captain

{118748: 'Mohamed Salah'}

## Evaluate potential transfers:

In [11]:
# Evaluate potential transfers from the currently selected squad:
from fpl_predictor.strategies.sample import evaluate_transfers, make_tranfers

# Change the criteria for ranking players:
sqb.cross_seasons = True
sqb.set_n(10)
sqb.set_scoring_metric("ict_index")

# Get suggested transfers:
transfers = evaluate_transfers(sqb, 2019, 7, live=True)
transfers

,available,code,minutes_percent,name,out_code,out_name,out_score,out_score_per_value,out_team,out_value,position,score,score_per_value,team,value,score_gain,score_per_val_gain,val_gain,squad_unavailable,first_team_unavailable
0,1.0,184341.0,0.500000,Mason Mount,172780.0,James Maddison,4.3,0.060563,Leicester,71.0,MID,10.660000,0.164000,Chelsea,65.0,6.360000,0.103437,6.0,True,True
1,1.0,94924.0,0.677778,Gerard Deulofeu,44699.0,Ashley Barnes,0.2,0.002985,Burnley,67.0,FWD,9.233333,0.148925,Watford,62.0,9.033333,0.145940,5.0,False,False
2,1.0,184341.0,0.500000,Mason Mount,61366.0,Kevin De Bruyne,4.8,0.048980,Man City,98.0,MID,10.660000,0.164000,Chelsea,65.0,5.860000,0.115020,33.0,False,False
3,1.0,101188.0,0.873333,Lucas Digne,206325.0,Oleksandr Zinchenko,4.8,0.085714,Man City,56.0,DEF,8.366667,0.137158,Everton,61.0,3.566667,0.051444,-5.0,False,False
4,1.0,126184.0,0.900000,Nathan Aké,149484.0,Tyrone Mings,1.8,0.039130,Aston Villa,46.0,DEF,5.133333,0.104762,Bournemouth,49.0,3.333333,0.065631,-3.0,False,False
5,1.0,126184.0,0.900000,Nathan Aké,153723.0,John Lundstram,2.5,0.059524,Sheffield Utd,42.0,DEF,5.133333,0.104762,Bournemouth,49.0,2.633333,0.045238,-7.0,False,False
6,1.0,37915.0,0.850000,Hugo Lloris,38533.0,Rui Pedro dos Santos Patrício,1.0,0.019608,Wolves,51.0,GK,3.577778,0.065051,Spurs,55.0,2.577778,0.045443,-4.0,False,False
7,1.0,126184.0,0.900000,Nathan Aké,39487.0,Erik Pieters,3.5,0.074468,Burnley,47.0,DEF,5.133333,0.104762,Bournemouth,49.0,1.633333,0.030294,-2.0,False,False
8,1.0,94924.0,0.677778,Gerard Deulofeu,57127.0,Teemu Pukki,9.0,0.128571,Norwich,70.0,FWD,9.233333,0.148925,Watford,62.0,0.233333,0.020353,8.0,False,False
9,1.0,80201.0,0.900000,Bernd Leno,98747.0,Nick Pope,3.1,0.067391,Burnley,46.0,GK,3.105556,0.062111,Arsenal,50.0,0.005556,-0.005280,-4.0,False,False


## Make Transfers:

In [11]:
sqb.squad.remove_player(172780)

Removed player from squad (14 remain): James Maddison


In [13]:
sqb.add_player(code=184341, year=2019, week=6, live=True)

Player added to squad: Mason Mount
